In [1]:
import openpyxl
import pyomo.environ as pe
import numpy as np
import pandas as pd

In [2]:
#################################################################################################################
# Objective: Read some appliance's characteristics from the excel "Info".
#
# ->Dev: Set of devices (1 a 15);
# ->Type: 0 (Not Flexibel), 1 (Power Flexibility), 2 (Time Flexibility) and 3 (Time and Power Flexibility);
# ->T_ON: Time that each load needs to be turned ON;
# ->T_OFF: Time that each load needs to be turned ON and OFF;
# ->State_init: Initial state (t=0) of each appliance: 0 (desligada) ou 1 (ligada);
# ->P_max: Maximum Power for each appliance.
# ->W_max: Maximum Weight for each appliance;
# ->W_min: Minimum Weight for each appliance;
# ->Y_max: Maximum Power percentage for each appliance;
# ->Y_min: Minimum Power percentage for each appliance;
# ->T_Hold_Power: Time needed to fix a power change for each appliance.
#
#################################################################################################################

# load the workbook
workbook = openpyxl.load_workbook('Info.xlsx')

# select the worksheet
worksheet = workbook['Lds_inputs']

# read 'Set of devices'
column = worksheet['B']
Dev = []
for cell in column:
    if cell.row == 1:
        continue
    Dev.append(cell.value)

# read 'Type of appliances'
column = worksheet['C']
Type = []
for cell in column:
    if cell.row == 1:
        continue
    Type.append(cell.value)

# read 'Time needed to be ON' 
column = worksheet['D']
T_ON = []
for cell in column:
    if cell.row == 1:
        continue
    T_ON.append(cell.value)

# read 'Time needed to be OFF' 
column = worksheet['E']
T_OFF = []
for cell in column:
    if cell.row == 1:
        continue
    T_OFF.append(cell.value)

# read 'Initial State'
column = worksheet['F']
State_init = []
for cell in column:
    if cell.row == 1:
        continue
    State_init.append(cell.value)

# read 'Maximum Power'
column = worksheet['G']
P_max = []
for cell in column:
    if cell.row == 1:
        continue
    P_max.append(cell.value)

# read 'Maximum Weight'
column = worksheet['H']
W_max = []
for cell in column:
    if cell.row == 1:
        continue
    W_max.append(cell.value)

# read 'Minimum Weight'
column = worksheet['I']
W_min = []
for cell in column:
    if cell.row == 1:
        continue
    W_min.append(cell.value)

# read 'Maximum percentage of power'
column = worksheet['J']
Y_max = []
for cell in column:
    if cell.row == 1:
        continue
    Y_max.append(cell.value)

# read 'Minimum percentage of power'
column = worksheet['K']
Y_min = []
for cell in column:
    if cell.row == 1:
        continue
    Y_min.append(cell.value)

for j in range(len(Dev)):
    Y_max [j] = Y_max [j] / 100
    Y_min [j] = Y_min [j] / 100

# read 'Time needed to fix a power change'
column = worksheet['L']
T_Hold_Power = []
for cell in column:
    if cell.row == 1:
        continue
    T_Hold_Power.append(cell.value)

# close the workbook
workbook.close()

# Print the values
print(f"Index of devices - {(Dev)}")
print(f"Type of appliances - {(Type)}")
print(f"T_ON - {(T_ON)}")
print(f"T_OFF - {(T_OFF)}")
print(f"Initial State - {(State_init)}")
print(f"Maximum Power - {(P_max)}")
print(f"Maximum Weight - {(W_max)}")
print(f"Minimum Weight - {(W_min)}")
print(f"Y_max - {(Y_max)}")
print(f"Y_min - {(Y_min)}")
print(f"T_Hold_Power - {(T_Hold_Power)}")



Index of devices - [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Type of appliances - [2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 1, 1]
T_ON - [6, 15, 7, 10, 38, 32, 50, 3, 10, 40, 40, 40]
T_OFF - [4, 4, 45, 40, 65, 60, 0, 0, 0, 0, 0, 0]
Initial State - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Maximum Power - [100, 300, 120, 120, 180, 2000, 192, 991, 1800, 250, 300, 350]
Maximum Weight - [10, 10, 10, 10, 10, 10, 10, 10, 10, 3, 3, 2]
Minimum Weight - [10, 10, 10, 10, 10, 10, 10, 10, 10, 1, 2, 1]
Y_max - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Y_min - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 0.8, 0.8]
T_Hold_Power - [-1, -1, -1, -1, -1, -1, -1, -1, -1, 10, 10, 10]


In [3]:
#################################################################################################################
# Objective: Declaration of sets, parameters and variables.
#
# ->Sets: T, Dev
# ->Parameters: P_lim
# ->Mutable Parameters: 
# ->Variables:
#
#################################################################################################################


# Sets and Constants:
T=12 # Set of indices in scheduling horizon        
P_lim = 3000 # Power limit      
rows = T # Auxiliary variable
cols = len(Dev) # Auxiliary variable 


# On/Off variables:
X_onoff_RT_vector = [0] * len(Dev) # Status of appliance j
X_onoff_RT_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
X_onoff_opt_RT_vector = [0] * len(Dev) # Optimized Status of appliance j


# Power change variables:
Y_RT_matrix = [[1 for j in range(cols)] for i in range(rows+1)] # Percentage of Power for each appliance j at time i 
Y_opt_RT_vector = [1] * len(Dev) # Percentage of Optimized Power for each appliance j


# Power variables:
P_RT = [0] * len(Dev) # Power of appliance j
P_sum=[0] * T # Sum of Power of all appliances at time i
P_opt_RT = [0] * len(Dev) # Optimized Power for each appliance j
P_opt_sum=[0] * T # Sum of Optimized Power of all appliances at time i


# Weight variables:
W_RT_vector = [10] * len(Dev) # Weight of appliance j
W_RT_matrix = [[10 for j in range(cols)] for i in range(rows)] # Weight of appliance j at time i
W_opt_RT_vector = [10] * len(Dev) # Optimized Weight of appliance j


# Auxiliary variables:
Optimization =0
X_onoff_before_opt_RT = [['' for j in range(cols)] for i in range(rows)]
cont_on_dataset = [0] * len(Dev)
cont_off_dataset = [0] * len(Dev)
cont_T_Hold_Power = [0] * len(Dev)
cont_on_weight = [0] * len(Dev)
cont_off_weight = [0] * len(Dev)
W_real_aux = [0] * len(Dev)
W_Power_aux = [0] * len(Dev)



#------------------------------------------------------#
# Duvido que seja necessário !!!!!!!!!!!!!!!!!!!!!!!!
weight_vector_before_optim = [['' for j in range(cols)] for i in range(rows)]
state = [0] * len(Dev) #(0=desligado, 1=ligado)
T_ON_Original = [0] * len(Dev)
T_ON_Original = T_ON.copy()
weight = [10] * len(Dev)
aux_optim = 0
force_turn_ON = 0
var_controlo = 0
T_ON_atualizado = [0] * len(Dev)
T_ON_atualizado = T_ON.copy()
control_var = [0] * len(Dev)


In [4]:
###########################################################################################################
# Objective: This function initializes the variables X_onoff_RT_vector and W_RT_vector.
#           ->Loads with time flexibility: 
#               If this type of loads starts ON, we have X=1 and W=10.
#               If this type of loads starts OFF, we have X=0 and W=1.
#           ->Loads non-flexible or loads with power flexibility: 
#               If this type of loads starts ON, we have X=1 and W=10.  
#               If this type of loads starts OFF, we have X=0 and W=10.   
#
# Output: Vector with the Status X of each load (0=OFF, 1=ON) at time i=0;
#         Vector with the Weight W of each load at time i=0.      
###########################################################################################################

def function_initialize(X_onoff_RT_vector, W_RT_vector):
    
    for j in range(len(Dev)): 
            
            # Loads with time flexibility
            if ((Type[j]==2 or Type[j]==3)):
                if (State_init[j] == 1): # Start ON   
                    X_onoff_RT_vector[j] = 1                       # State ON
                    W_RT_vector[j] = 10 - (9/T_ON_atualizado[j])   # Weight
                    
                else: # Start OFF
                    X_onoff_RT_vector[j] = 0                       # State OFF
                    W_RT_vector[j] = 1 + (9/T_OFF[j])              # Weight

            # Loads non-flexible
            # Loads with power flexibility
            if Type[j] == 0 or Type[j] == 1:
                if (State_init[j] == 1):  # Start ON         
                    X_onoff_RT_vector[j] = 1                       # State ON
                   
                else: # Start OFF         
                    X_onoff_RT_vector[j] = 0                       # State ON
                
                W_RT_vector[j] = 10                                # Weight
    
    return X_onoff_RT_vector, W_RT_vector

               

In [5]:

###########################################################################################################
# Objetive: Form a dataset that will work as an input for the optimization problem.
#
#           -> Loads with time flexibility: 
#              If the previous weight is lower than 1, the load is turned off;
#              If the previous weight is higher than 10, the load is turned on;
#           -> Loads with power flexibility and non-flexible loads:
#              Since the time component of the weight of this type of loads is always 10, 
#              the status of each load is decided through T_ON, T_OFF and counters;
#              Note: I am going to assume the following:
#                   - Loads with power flexibility: Turned on at 07:00 , 19:00 , 20:00 and 21:00;
#                   - Non-flexible loads: Turned on at 13:00 and 20:00;
#
# Output: Vector with the Status X of each load (0=OFF, 1=ON).
###########################################################################################################

def function_dataset(i, X_onoff_RT_vector, W_RT_vector):
    
    for j in range(len(Dev)):

        # Loads with time flexibility
        if (Type[j]==2 or Type[j]==3):

            if X_onoff_RT_vector[j] == 1: # State ON, from Weight=10 to Weight=1
                if W_RT_vector[j] <= 1 and W_RT_vector[j] != -1: # Weight=-1 means that the load needs to be turned on                    
                    X_onoff_RT_vector[j] = 0 # Turn off the load

            elif X_onoff_RT_vector[j] == 0: # State OFF, from Weight=1 to Weight=10              
                if W_RT_vector[j] >= 10 or W_RT_vector[j] == -1:                    
                    X_onoff_RT_vector[j] = 1 # Turn on the load

        # Loads non-flexible
        # Loads with power flexibility
        if (Type[j]== 0 or Type[j]== 1):

            if X_onoff_RT_vector[j] == 1: # State ON                              
                if cont_on_dataset[j] < T_ON_atualizado[j]:                    
                    cont_on_dataset[j] += 1
                else:
                    X_onoff_RT_vector[j] = 0 # Turn off the load                                     
                    cont_on_dataset[j] = 0
                    
            elif X_onoff_RT_vector[j] == 0: # State OFF                        
                if ((Type[j]== 1) and (i==7 or i==19*60 or i==20*60 or i==21*60)) or ((Type[j]== 0) and (i==13 or i==20*60)): 
                #Turn on the loads with power flexibility at 07:00 , 19:00 , 20:00 and 21:00; 
                #Turn on the loads non-flexible at 13:00 and 20:00;
                    X_onoff_RT_vector[j] = 1 # Turn on the load
                    cont_on_dataset[j] += 1

    return (X_onoff_RT_vector)
    

In [6]:
#######################################################################################################################
# Objective: Provide weights relatively to the time component for each load to represent the comfort level of the equipment.
#
#           1) Weights for each load before optimization;
#           2) Weights for each load after optimization;
# 
#           -> If the load is ON, the weight decreases from 10 to 1 (priority decreases, the load can be turned OFF);  
#           -> If the load is OFF, the weight increases from 1 to 10 (priority increases, the load must be turned ON);       
#           
#           Note: Since we are working in real time, we only want to cut loads that are ON with priorities close to 1,
#           so the weights calculated when the loads are OFF are not relevant.
#       
# Output: Vector with the time component of weights for each load.    
#######################################################################################################################

def function_weight (i, Optimization, X_onoff_RT_vector, X_onoff_opt_RT_vector, W_RT_matrix):
   
    if Optimization == 0: # Before Optimization: 
       
        for j in range(len(Dev)): 

            # Loads with time flexibility
            if (Type[j]==2 or Type[j]==3):
                if W_RT_matrix[i-1][j]!=20 and W_RT_matrix[i-1][j]!=-1: # When W=20 or W=-1, the system don't change it
                    if X_onoff_RT_vector[j] == 1 : # State ON                                                                                               
                        W_RT_vector[j] =  W_RT_matrix [i-1][j] - (9/T_ON_atualizado[j] ) # Weight decreases
                    if X_onoff_RT_vector[j] == 0 : # State OFF 
                        W_RT_vector[j] =  W_RT_matrix [i-1][j] + (9/T_OFF[j] ) # Weight increases
      
            W_RT_vector[j]=round(W_RT_vector[j], 3)
        return (W_RT_vector)


    elif Optimization == 1: # After Optimization: 
       
        for j in range(len(Dev)): 
            
            # Loads with time flexibility
            # If the optimization turned off the load
            if (Type[j]==2 or Type[j]==3) and (X_onoff_RT_vector[j]==1 and X_onoff_opt_RT_vector[j])==0:  
                if  W_RT_vector[j] != 20 and i!=0:
                    W_RT_vector[j] = W_RT_matrix[i-1][j] + 9/T_OFF[j] # Weight increases
                
                if (W_RT_vector[j]>10 and W_RT_vector[j]!=20):
                    W_real_aux[j] = W_RT_vector[j]
                    W_RT_vector[j] = 20 # Weight=20 means that the load needs to be turned on
            
                elif (W_RT_vector[j] == 20):
                    W_real_aux[j] = W_real_aux[j] + 9/T_OFF[j]
                    W_RT_vector[j] = -1 # Weight=-1 means that the load have to be turned on

            # Loads non-flexible
            # Loads with power flexibility
            if Type[j] == 0 or Type[j] == 1:
                W_RT_vector[j] = 10
            
        return (W_RT_vector)
             

In [7]:
#######################################################################################################################
# Objective: Sum the power for each load at time i.
#
#           1) Before optimization, X= X before optimization;
#           2) After optimization, X= X optimized;
#       
# Output: Value with the sum of power at time i.    
#######################################################################################################################

def function_P_sum (X_onoff_RT_vector, P_RT):
    
    P_sum = 0
    
    for j in range(len(Dev)):     
        if X_onoff_RT_vector[j] == 1: # State ON
            P_sum = P_sum + P_RT[j]
    
    return(P_sum)


In [8]:
#######################################################################################################################
# Objective: Fix the change of power during T_Hold_Power.  
#            ->If the power(i) is different from power(i-1), the counter is incremented;
#            ->If the counter is equal to T_Hold_Power and the power is not 100%, the power is change to the maximum,
#              so that in next event the optimization try to keep it in 100%;    
# Output: Vector with the percentage of power of each load at time i.    
#######################################################################################################################

def function_fix_y (i, X_onoff_RT_vector, Y_RT_matrix):
   
    for j in range(len(Dev)):
        
        # Loads with power flexibility
        if (Type[j] == 1 or Type[j] == 3):

            if X_onoff_RT_vector[j] == 1: # State ON

                if Y_RT_matrix[i][j] != Y_RT_matrix[i-1][j] and cont_T_Hold_Power [j]==0:
                    cont_T_Hold_Power [j] += 1
                
                elif cont_T_Hold_Power [j]!=0 and cont_T_Hold_Power [j]< T_Hold_Power [j]:
                    cont_T_Hold_Power [j] += 1

                    if cont_T_Hold_Power [j] == T_Hold_Power [j] and Y_opt_RT_vector[j] != 1:
                        Y_RT_matrix [i][j] = 1 # Change the power to the maximum
                        cont_T_Hold_Power [j] = 0 
                    #if cont_T_Hold_Power [j] == T_Hold_Power [j] and Y_opt_RT_vector[j] == 1:
                       # Y_RT_matrix [i][j] = 1 # Change the power to the maximum
                       # cont_T_Hold_Power [j] = T_Hold_Power [j]  
                    
            else: # State OFF
                Y_RT_matrix [i][j] = 1 # Change/keep the power to the maximum
                cont_T_Hold_Power [j] = 0
                
    return(Y_RT_matrix [i], cont_T_Hold_Power)

In [9]:
######################################################################################################################
# Optimization: Maximize the user's comfort.
#
# Constraint 1: The sum of each load's power has to be lower than the threshold;
# Constraint 2: The power's percentage that can be decreased has to be higher than the minimum;
# Constraint 3: The power's percentage that can be decreased has to be lower than the maximum;
# Constraint 4: The status of loads with power flexibility, non-flexible, or weight=-1 can not be changed; 
#               The status of loads with time flexibility and weight!=-1 can be changed: 
#                   -If it is on, it can keep or turn off;
#                   -If it is off, it just can keep turned off;
#
# Constraint 5: If the power is not the same as last time i and the load is turned on, the optimization must keep it.
#
# Outputs:  - Maximum comfort; 
#           - Vetor of load's status, X_opt; 
#           - Vetor with load's power percentage, Y_opt;
#           - The power that exceeds the limit, P_rlx.
######################################################################################################################

model = pe.ConcreteModel()

# Set Declaration
model.loads = pe.Set(initialize=np.arange(len(Dev)))

# Parameter Declaration
model.lim = pe.Param(initialize=P_lim)
model.P_max = pe.Param(range(len(Dev)),initialize=P_max)
model.T_Hold_Power = pe.Param(range(len(Dev)),initialize=T_Hold_Power)
model.W_time =pe.Param(range(len(Dev)),initialize=0, mutable=True)
model.W_max = pe.Param(range(len(Dev)),initialize=W_max)
model.W_min = pe.Param(range(len(Dev)),initialize=W_min)
model.Y_max = pe.Param(range(len(Dev)),initialize=Y_max)
model.Y_min = pe.Param(range(len(Dev)),initialize=Y_min)
model.X =pe.Param(range(len(Dev)),initialize=0, mutable=True)
model.Y =pe.Param(range(len(Dev)),initialize=1, mutable=True)
model.cont_T_Hold_Power =pe.Param(range(len(Dev)),initialize=0, mutable=True)

# Variable Declaration
model.X_opt = pe.Var(range(len(Dev)),domain=pe.Binary, initialize={i: 1 for i in range(len(Dev))})
model.Y_opt = pe.Var(range(len(Dev)),domain=pe.NonNegativeReals, initialize={i: 1 for i in range(len(Dev))})
model.P_rlx = pe.Var(domain=pe.NonNegativeReals, initialize=0)

# Constrait 1
def _constraint_one(m):
   return sum((m.P_max[j]*m.X_opt[j]*m.Y_opt[j]) for j in range(len(Dev))) <= m.lim + m.P_rlx
model.constraint_one = pe.Constraint(rule=_constraint_one)

# Constrait 2
def _constraint_two(m, j):
        return ((m.Y_min[j] <= m.Y_opt[j]))
model.constraint_two = pe.Constraint(model.loads, rule=_constraint_two)

# Constrait 3
def _constraint_three(m, j):
    return ((m.Y_opt[j] <= m.Y_max[j]))
model.constraint_three = pe.Constraint(model.loads, rule=_constraint_three)

# Objective function
def _objective_function(m):
    return ( sum(m.W_time[j]*m.X_opt[j]+ ((m.W_min[j] + ((m.W_max[j] - m.W_min[j]) / (m.Y_max[j] - m.Y_min[j] + 0.1)) * (m.Y_opt[j] - m.Y_min[j]))*m.X_opt[j]) for j in range(len(Dev))) ) - (10000*m.P_rlx)
model.objective = pe.Objective(expr=_objective_function, sense=pe.maximize)


In [10]:
###########################################################################################################
# Objetivo: Resolve problema em Real-Time
# Funções usadas: 
#                   ->function_weight (dá weight_auxs e faz optimizations dos mesmos);
#                   ->function_sum (sum as potencias de cada carga para ti);
#                   ->optimização (decide quais cargas estão ON ou OFF, maximizando o conforto);
# Input: Informação de cada carga, dataset com ON/OFF.
# Output: Optimização do dataset.
###########################################################################################################

for i in range (T): 

    # Before Optimization
    Optimization=0 
    
    if i==0: # Vectors Initialization
        X_onoff_RT_vector, W_RT_vector = function_initialize(X_onoff_RT_vector, W_RT_vector)
        X_onoff_RT_matrix[i] = X_onoff_RT_vector.copy() # Status on/off of loads
        W_RT_matrix [i] = W_RT_vector.copy() # Weight of loads
        
    else: 
        # Status on/off of loads
        X_onoff_RT_vector = function_dataset (i, X_onoff_RT_vector, W_RT_vector)                                
        X_onoff_RT_matrix[i] = X_onoff_RT_vector.copy()
        
        # Weight of loads
        W_RT_vector = function_weight(i, Optimization, X_onoff_RT_vector, X_onoff_opt_RT_vector, W_RT_matrix) 
        W_RT_matrix [i] = W_RT_vector.copy()

    # Sum of load's power
    P_sum[i] = function_P_sum(X_onoff_RT_vector, P_RT)                                             

    if P_sum[i] > P_lim: # Check the need for optimization
        
        # Initialization of some optimization parameters
        for j in range(len(Dev)):                                                       
            model.W_time[j] = W_RT_vector[j]  # Time Component of Weight
            model.X[j] = X_onoff_RT_vector[j] # Status on/off of each loads
            model.Y[j] = Y_opt_RT_vector[j]   # Percentage of each load's power
            model.cont_T_Hold_Power[j] = cont_T_Hold_Power[j] # Counter of T_Hold

        # Constraint 4
        def _constraint_four(m,j):  
            # Loads non-flexible
            # Loads with power flexibility
            if Type[j] == 0 or Type[j] == 1 or W_RT_vector[j] == -1: 
                return m.X_opt[j] == m.X[j] # The optimization status on/off must be the same as the before optimization
            # Loads with time flexibility
            if (Type[j] == 2 or Type[j] == 3) and W_RT_vector[j] != -1:
                return m.X_opt[j] <= m.X[j] # The optimization can keep or turn off the load's status
            
            #RETIRARRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
            #else:
                #return (pe.Constraint.Feasible)
        model.constraint_four = pe.Constraint(model.loads, rule=_constraint_four)

        # Constraint 5
        def _constraint_five(m,j): 
            # Change of power 
            if cont_T_Hold_Power[j]!=0: #and cont_T_Hold_Power[j] != m.T_Hold_Power [j]: 
                return (m.Y_opt[j] ==  m.Y[j]) # The optimization percentage of each load's power must be the same as the before optimization 
            # Power is maintained until T_Hold

            #PORQUE É PRECISOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
            else:
                return (pe.Constraint.Feasible) # The optimization change the percentage of power
        model.constraint_five = pe.Constraint(model.loads, rule=_constraint_five)

        # Solver                                                 
        results = pe.SolverFactory('scip', executable='C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe').solve(model)
        #model.pprint()
        
        # Save the results before optimization
        X_onoff_before_opt_RT[i] = X_onoff_RT_vector.copy()
        weight_vector_before_optim[i] = W_RT_vector.copy()

        # After Optimization
        Optimization=1 

        # Save the optimization results
        for j in range(len(Dev)):
            X_onoff_opt_RT_vector [j] = int (np.abs(model.X_opt[j].value)) # Status on/off of each loads after optimization
            if X_onoff_opt_RT_vector [j] == 1:
                Y_opt_RT_vector [j] = (model.Y_opt[j].value) # Percentage of each load's power after optimization
            P_opt_RT[j] = (model.P_max[j]*model.X_opt[j].value*model.Y_opt[j].value) # Power of each load after optimization
            # Power component of weight
            W_Power_aux [j] = ((W_min[j] + ((W_max[j] - W_min[j]) / (Y_max[j] - Y_min[j] + 0.1)) * (Y_opt_RT_vector [j] - Y_min[j]))*X_onoff_opt_RT_vector [j])

        Y_RT_matrix[i]= Y_opt_RT_vector # Percentage of each load's power after optimization
        Objective_Function_RT = model.objective() # Result of objective function
    
        # Print of Results
        
        print(f"\nPeriodo {i}: Há optimização\n")
        print(f"TIME")
        print(f"ON/OFF before optim = {(X_onoff_RT_vector)}")
        print(f"x[] = {(X_onoff_opt_RT_vector)}")
        print(f"weight_time[] = {(W_RT_vector)}")
        print(f"weight_potencia[] = {(W_Power_aux)}")
        print(f"Power_max[] = {(P_max)}\n")
        #print(f"Peso model = {(model.weight_aux[].value)}")
        #print(f"weight_time[] = {(weight)}")
        print(f"POWER")
        print(f"y[] = {(Y_opt_RT_vector)}")
        print(f"Power_after_optim[] = {(P_opt_RT)}\n")
        print(f"Objective Funtion")
        print(f"Maximizar OF = {model.objective()}")
                        
        # Update the status on/off of loads
        X_onoff_RT_vector = X_onoff_opt_RT_vector.copy()
        X_onoff_RT_matrix[i] = X_onoff_RT_vector.copy() 

        # Update the weight of loads                                                 
        W_RT_vector = function_weight(i, Optimization, X_onoff_before_opt_RT[i], X_onoff_opt_RT_vector, W_RT_matrix)
        W_RT_matrix [i] = W_RT_vector.copy()

        # Update the power of loads
        P_opt_sum[i] = function_P_sum(X_onoff_RT_vector, P_RT)                                     


    # Update the weight of loads 
    for j in range(len(Dev)):
        if (Type[j]==2 or Type[j]==3):
            if (W_RT_vector[j] == 20 or W_RT_vector [j] == -1) and X_onoff_RT_vector[j] == 1:
                W_RT_vector[j] = W_real_aux[j] - 9/T_ON_atualizado[j]
                W_real_aux[j] = 0
    W_RT_matrix [i] = W_RT_vector.copy()

    Y_opt_RT_vector, cont_T_Hold_Power = function_fix_y (i, X_onoff_RT_vector, Y_RT_matrix)
    Y_RT_matrix [i] = Y_opt_RT_vector.copy()    

    #Vector Power and T_ON Update
    for j in range(len(Dev)):
        P_RT[j] = P_max [j]*Y_opt_RT_vector[j]
        if Type[j] == 3: #  Loads with time and power flexibility
            T_ON_atualizado[j] = (P_max[j]*T_ON[j]) / P_RT[j]



In [11]:
#################################################################################################################
# Objective: ->Add labels for a better understanding of the results.
#            ->Add the minutes and indexes to the matrix of the status on/off and weights.
#            ->Add the sum of load's power before and after the optimization to the matrix of status on/off.
#################################################################################################################

col = []
for i in range(T):
    col.append(i)

for i in range(T):
    X_onoff_RT_matrix[i].insert(0, col[i])
    X_onoff_before_opt_RT[i].insert(0, col[i])
    W_RT_matrix[i].insert(0, col[i])
    weight_vector_before_optim[i].insert(0, col[i])
    
for i in range(T):
    X_onoff_RT_matrix[i].append(0)
    X_onoff_RT_matrix[i].append(0)
    #change value in ON_or_OFF list in position 16 for sum[i]
    X_onoff_RT_matrix[i][13] = P_sum[i]
    X_onoff_RT_matrix[i][14] = P_opt_sum[i]
    
row = []
for i in range(len(Dev)+3):
    row.append(i)
row [0] = 'Min/Index'
row [13] ='Sum Power'
row [14] ='Power Optim'
row1 = []
for i in range(len(Dev)+1):
    row1.append(i)
row1 [0] = 'Min/Index'

X_onoff_RT_matrix.insert(0, row)
X_onoff_before_opt_RT.insert(0, row1)
W_RT_matrix.insert(0, row1)
weight_vector_before_optim.insert(0, row1)

In [12]:
############################################################################################################
# Objectivo: Imprime as 4 worksheets.
# Função: Escreve no excel: 
#               ->tabela com ON/OFF optimizado para cada load para ti;
#               ->tabela com ON/OFF antes da optimização;
#               ->tabela com weight_auxs optimizados;
#               ->tabela com weight_auxs antes da optimização;
############################################################################################################


# Create a Pandas Excel writer using XlsxWriter
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')

df = pd.DataFrame(X_onoff_RT_matrix)
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Dataset', index=False, header=False)

df2 = pd.DataFrame(X_onoff_before_opt_RT)
df2.to_excel(writer, sheet_name='Dataset before', index=False, header=False)

df3 = pd.DataFrame(W_RT_matrix)
df3.to_excel(writer, sheet_name='weight_vector_RT', index=False, header=False)

df4 = pd.DataFrame(weight_vector_before_optim)
df4.to_excel(writer, sheet_name='weight_vector_RT before', index=False, header=False)

writer.save()

writer.close()



C:\Users\anaol\AppData\Local\Temp\ipykernel_15004\2554789802.py:27: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\anaol\AppData\Local\Programs\Python\Python311\Lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
